In [0]:
import pandas as pd
import numpy as np
train = pd.read_csv("train_prep.csv")
test = pd.read_csv("test_prep.csv")

In [3]:
train.drop(['APARTMENT.NUMBER', 'LAND.SQUARE.FEET', 'GROSS.SQUARE.FEET', 'SALE.DATE', 'ZIP.CODE'], axis=1, inplace=True)
test.drop(['APARTMENT.NUMBER', 'LAND.SQUARE.FEET', 'GROSS.SQUARE.FEET', 'SALE.DATE'], axis=1, inplace=True)
train.isna().sum()

NEIGHBORHOOD                        0
BUILDING.CLASS.CATEGORY             0
TAX.CLASS.AT.PRESENT              182
BLOCK                               0
LOT                                 0
BUILDING.CLASS.AT.PRESENT         182
RESIDENTIAL.UNITS                   0
COMMERCIAL.UNITS                    0
TOTAL.UNITS                         0
YEAR.BUILT                          0
TAX.CLASS.AT.TIME.OF.SALE           0
BUILDING.CLASS.AT.TIME.OF.SALE      0
sale_year                           0
sale_month                          0
sale_day                            0
sale_minus_built                    0
high.end                            0
dtype: int64

In [4]:
train = train.dropna()
# test = test.dropna()
train.isna().sum()

NEIGHBORHOOD                      0
BUILDING.CLASS.CATEGORY           0
TAX.CLASS.AT.PRESENT              0
BLOCK                             0
LOT                               0
BUILDING.CLASS.AT.PRESENT         0
RESIDENTIAL.UNITS                 0
COMMERCIAL.UNITS                  0
TOTAL.UNITS                       0
YEAR.BUILT                        0
TAX.CLASS.AT.TIME.OF.SALE         0
BUILDING.CLASS.AT.TIME.OF.SALE    0
sale_year                         0
sale_month                        0
sale_day                          0
sale_minus_built                  0
high.end                          0
dtype: int64

In [5]:

test['TAX.CLASS.AT.PRESENT'].fillna(2, inplace=True)
test['BUILDING.CLASS.AT.PRESENT'].fillna(2, inplace=True)
test.isna().sum()

NEIGHBORHOOD                      0
BUILDING.CLASS.CATEGORY           0
TAX.CLASS.AT.PRESENT              0
BLOCK                             0
LOT                               0
BUILDING.CLASS.AT.PRESENT         0
ZIP.CODE                          0
RESIDENTIAL.UNITS                 0
COMMERCIAL.UNITS                  0
TOTAL.UNITS                       0
YEAR.BUILT                        0
TAX.CLASS.AT.TIME.OF.SALE         0
BUILDING.CLASS.AT.TIME.OF.SALE    0
sale_year                         0
sale_month                        0
sale_day                          0
sale_minus_built                  0
ID                                0
dtype: int64

In [6]:
train.head(20)

,NEIGHBORHOOD,BUILDING.CLASS.CATEGORY,TAX.CLASS.AT.PRESENT,BLOCK,LOT,BUILDING.CLASS.AT.PRESENT,RESIDENTIAL.UNITS,COMMERCIAL.UNITS,TOTAL.UNITS,YEAR.BUILT,TAX.CLASS.AT.TIME.OF.SALE,BUILDING.CLASS.AT.TIME.OF.SALE,sale_year,sale_month,sale_day,sale_minus_built,high.end
0,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,A4,True,False,True,1860,1,A4,2019,9,25,159,0
1,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404,1,B9,True,False,True,1900,1,B9,2019,7,22,119,0
2,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404,1,B9,True,False,True,1900,1,B9,2018,11,29,118,0
3,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404,1,B9,True,False,True,1900,1,B9,2018,11,29,118,0
4,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,372,23,C1,True,False,True,2001,2,C1,2019,8,8,18,0
5,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,376,41,C4,True,False,True,1900,2,C4,2019,9,12,119,0
6,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,387,119,C7,True,True,True,1920,2,C7,2019,2,27,99,0
7,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,390,50,C1,True,False,True,1900,2,C1,2018,11,30,118,0
8,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,393,23,C7,True,True,True,1900,2,C7,2018,12,5,118,1
9,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,401,15,C5,True,True,True,1900,2,C5,2018,12,12,118,1


In [0]:
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from random import seed
import random
seed(22)
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

num_attribs = ['YEAR.BUILT', 'sale_year', 'sale_day', 'sale_minus_built', 'LOT']
cat_attribs = ['BUILDING.CLASS.CATEGORY', 'NEIGHBORHOOD','TAX.CLASS.AT.PRESENT','BUILDING.CLASS.AT.TIME.OF.SALE', 
                'RESIDENTIAL.UNITS', 'COMMERCIAL.UNITS','BUILDING.CLASS.AT.PRESENT', 
               'TOTAL.UNITS', 'TAX.CLASS.AT.TIME.OF.SALE', 'sale_month', 'BLOCK']

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_attribs),
    ])


x_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]
y_train = pd.DataFrame({"high_end":y_train})
x_test = test.iloc[:,:-1]
id_test = test.iloc[:,-1]
id_test = pd.DataFrame({"id":id_test})

# transform data using pipeline
x_train = full_pipeline.fit_transform(x_train)
x_test = full_pipeline.transform(x_test)

In [22]:
# number of input variables
np.size(x_train,1)

1675

In [16]:
# https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
# https://keras.io/getting-started/sequential-model-guide/#mlp-for-binary-classification

import keras
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation
from keras.constraints import maxnorm
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
# construct f1 metric for classifier to see how well it's doing
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

classifier = Sequential()
# classifier.add(Dropout(0.2, input_shape=(207,)))
classifier.add(Dense(512, activation='relu', kernel_initializer='random_normal', input_dim=1675, kernel_constraint=maxnorm(15)))
# classifier.add(Dropout(0.3))
# classifier.add(Dense(512, activation='relu', kernel_initializer='random_normal', kernel_constraint=maxnorm(5)))
# classifier.add(Dropout(0.5))
classifier.add(Dense(256, activation='relu', kernel_initializer='random_normal', kernel_constraint=maxnorm(15)))
# classifier.add(Dropout(0.3))
classifier.add(Dense(256, activation='relu', kernel_initializer='random_normal', kernel_constraint=maxnorm(15)))
# classifier.add(Dropout(0.1))
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', kernel_constraint=maxnorm(15)))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
# sgd = SGD(lr=0.1, decay=1e-2, momentum=0.9)

#Compiling the neural network
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics =[f1_m, precision_m, recall_m, 'accuracy'])

#Fitting the data to the training dataset 87
classifier.fit(x_train, final_y_train, batch_size=20, epochs=70)

eval_model=classifier.evaluate(x_train, final_y_train)
eval_model

Epoch 1/70
7044/7044 [==============================] - 8s 1ms/step - loss: 0.4889 - f1_m: 0.2815 - precision_m: 0.3978 - recall_m: 0.2440 - acc: 0.7618
Epoch 2/70
7044/7044 [==============================] - 6s 917us/step - loss: 0.3799 - f1_m: 0.5975 - precision_m: 0.7489 - recall_m: 0.5437 - acc: 0.8248
Epoch 3/70
7044/7044 [==============================] - 7s 932us/step - loss: 0.3199 - f1_m: 0.6776 - precision_m: 0.7754 - recall_m: 0.6453 - acc: 0.8509
Epoch 4/70
7044/7044 [==============================] - 7s 944us/step - loss: 0.2635 - f1_m: 0.7463 - precision_m: 0.8310 - recall_m: 0.7155 - acc: 0.8815
Epoch 5/70
7044/7044 [==============================] - 7s 946us/step - loss: 0.2171 - f1_m: 0.7877 - precision_m: 0.8552 - recall_m: 0.7663 - acc: 0.8999
Epoch 6/70
7044/7044 [==============================] - 7s 929us/step - loss: 0.1780 - f1_m: 0.8304 - precision_m: 0.8727 - recall_m: 0.8216 - acc: 0.9204
Epoch 7/70
7044/7044 [==============================] - 6s 921us/step - 

[0.025296360539783416,
 0.8995274444116509,
 0.8950466828072768,
 0.9100968396372581,
 0.9866553094832482]

In [0]:
y_pred=classifier.predict(x_test) # 0.90

In [0]:
y_pred.shape
y_pred = np.round(y_pred, 0)

In [19]:
df = pd.DataFrame(np.hstack((y_pred)), 
                  columns=['high.end'])
print(df.head(60))
df['ID'] = np.arange(len(df)) + 1

    high.end
0        1.0
1        0.0
2        0.0
3        1.0
4        0.0
5        1.0
6        1.0
7        0.0
8        1.0
9        1.0
10       1.0
11       1.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       1.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       1.0
28       0.0
29       0.0
30       1.0
31       0.0
32       1.0
33       0.0
34       0.0
35       0.0
36       0.0
37       0.0
38       0.0
39       0.0
40       0.0
41       0.0
42       0.0
43       0.0
44       0.0
45       0.0
46       0.0
47       0.0
48       0.0
49       0.0
50       1.0
51       0.0
52       0.0
53       0.0
54       0.0
55       0.0
56       0.0
57       0.0
58       0.0
59       0.0


In [0]:
from google.colab import files
df.to_csv('entry13.csv') 